

Olá estudante!

Me chamo Rafael Meirelles e irei revisar o seu projeto hoje e em eventuais futuras submissões até que ele cumpra todos os requisitos para o aceite.
Conte comigo nessa jornada e não se preocupe se precisar ajustar alguns detalhes, é parte do processo e fundamental para que você exercite os conceitos que vem aprendendo e assim melhore a qualidade dos seus códigos e análises.

**Peço que mantenha e não altereos comentários que eu fizer por aqui para que possamos nos localizar posteriormente, ok?**

Mais uma coisa, vamos utilizar um código de cores para você entender os meus feedbacks no seu notebook. Funciona assim:


<div class="alert alert-danger">
<strong>Vermelho</strong>

Erro que precisa ser consertado, caso contrário, seu projeto não pode ser aceito

</div>

<div class="alert alert-warning">
<strong>Amarelo</strong>

Alerta de um erro não crítco, mas que pode ser corrigido para melhoria geral no seu código/análise

</div>

<div class="alert alert-success">
<strong>Verde</strong>

Elogios
</div>

<div class="alert alert-info">
<strong>Comentário do estudante</strong>

Use uma caixa azul como essa para eventuais comentários que você gostaria de fazer para mim.
</div>






<div class="alert alert-danger">
<strong>Comentário do revisor v1</strong>

Estamos no caminho certo com essa versão, mas retocar os pontos indicados abaixo nos fará chegar em um resultado ainda mais incrível.
    
Conte comigo para ajudá-lo (a) em qualquer questão que possa vir a ter! Lembre-se de usar as caixas azuis para se comunicar comigo.
    
Até breve.
</div>


In [11]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
db_config = {
'user': 'practicum_student', # username
'pwd': 'QnmDH8Sc2TQLvy2G3Vvh7', # password
'host': 'yp-trainers-practicum.cluster-czs0gxyx2d8w.us-east-1.rds.amazonaws.com', 
'port': 5432, # connection port
'db': 'data-analyst-final-project-db' # the name of the database
}
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
db_config['pwd'],
db_config['host'],
db_config['port'],
db_config['db'])
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

Autor: Thaynnan Aurick

Data: 15-05-2000

# Encontre o número de livros lançados depois de 1 de janeiro de 2000.

In [12]:
query = """
SELECT 
    EXTRACT(YEAR FROM publication_date) AS ano,
    COUNT(*) AS livros_lancados,
    ROUND(AVG(num_pages)) AS media_paginas,
    ROUND(AVG(rating), 2) AS media_avaliacao
FROM 
    books b
LEFT JOIN 
    (SELECT book_id, AVG(rating) AS rating FROM ratings GROUP BY book_id) r
    ON b.book_id = r.book_id
WHERE 
    publication_date > '2000-01-01'
GROUP BY 
    ano
ORDER BY 
    ano DESC;
"""
total_livros = pd.io.sql.read_sql(query, con=engine)
quantidade_de_livros = total_livros['livros_lancados'].sum()
print(f"\nTOTAL DE LIVROS LANÇADOS APÓS 01/01/2000: {quantidade_de_livros}\n")
total_livros


TOTAL DE LIVROS LANÇADOS APÓS 01/01/2000: 819



,ano,livros_lancados,media_paginas,media_avaliacao
0,2020.0,1,360.0,4.50
1,2019.0,2,237.0,3.50
2,2014.0,2,290.0,4.50
3,2013.0,1,249.0,3.90
4,2012.0,3,763.0,4.29
5,2011.0,1,263.0,4.33
6,2010.0,7,289.0,4.11
7,2009.0,6,413.0,4.14
8,2008.0,4,426.0,3.53
9,2007.0,50,365.0,3.93


* Na tabela acima notamos que os anos com maior numero de lançamentos foi nos anos de:
* 2006 com 184 livros.
* 2005 com 139 livros.
* 2004 com 124 livros.
* E com um total de 819 livros lançados após 01/01/2000


<div class="alert alert-danger">
<strong>Comentário do revisor v1</strong>

A direção é essa mesmo, Thaynnan, mas inmportante dar uma resposta concreta por aqui, quantos livros foram lançados pós 1/1/2000?

</div>

<div class="alert alert-success">
<strong>Comentário do revisor v2</strong>

Valeu pelo ajuste!
</div>


# Livros publicados por editoras.

In [13]:
query = """
SELECT 
    p.publisher,
    COUNT(b.book_id) AS total_livros
FROM 
    publishers p
JOIN 
    books b ON p.publisher_id = b.publisher_id
GROUP BY 
    p.publisher
ORDER BY 
    total_livros DESC;
"""
pd.io.sql.read_sql(query, con=engine)

,publisher,total_livros
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19
...,...,...
335,See Sharp Press,1
336,Pocket Star,1
337,W.W. Norton & Company,1
338,Penguin Books Limited,1


* Podemos notar que o maximo de livros por editora foram de 42 e o minimo foi 1.


<div class="alert alert-success">
<strong>Comentário do revisor v1</strong>

A questão está correta!

</div>

# Numero de avaliações e Classificação media.

In [14]:
query = """
SELECT 
    b.book_id,
    b.title,
    ROUND(AVG(r.rating), 2) AS nota_media,
    COUNT(r.rating_id) AS total_avaliacoes,
    RANK() OVER (ORDER BY AVG(r.rating) DESC) AS classificacao
FROM 
    books b
LEFT JOIN 
    ratings r ON b.book_id = r.book_id
GROUP BY 
    b.book_id, b.title
HAVING 
    COUNT(r.rating_id) >= 1  -- Considera apenas livros com pelo menos 1 avaliação
ORDER BY 
    nota_media DESC;
"""

resultado = pd.io.sql.read_sql(query, con=engine)
resultado.head()  # Mostra os top 10 livros melhor avaliados

,book_id,title,nota_media,total_avaliacoes,classificacao
0,518,Pop Goes the Weasel (Alex Cross #5),5.0,2,1
1,732,The Ghost Map: The Story of London's Most Terr...,5.0,2,1
2,347,In the Hand of the Goddess (Song of the Liones...,5.0,3,1
3,610,Tai-Pan (Asian Saga #2),5.0,2,1
4,330,How to Be a Domestic Goddess: Baking and the A...,5.0,2,1


# Identifique a editora que lançou o maior número de livros com mais de 50 páginas.

query = """
SELECT 
    p.publisher,
    COUNT(b.book_id) AS total_livros,
    MAX(b.num_pages) AS max_paginas,
    ROUND(AVG(b.num_pages)) AS media_paginas
FROM 
    publishers p
JOIN 
    books b ON p.publisher_id = b.publisher_id
WHERE 
    b.num_pages > 300  -- Filtro para livros mais extensos
GROUP BY 
    p.publisher
ORDER BY 
    total_livros DESC
LIMIT 5;
"""
pd.io.sql.read_sql(query, con=engine)

* A Penguin Books logicamente ficaria com maior número de livros com muitas paginas ja que e a editora com a maior quantidade de livros publicados. 

# Identifique o autor com a média mais alta de classificação de livros, apenas para livros com pelo menos 50 classificações

In [15]:
query = """
WITH livros_avaliados AS (
    SELECT 
        b.book_id,
        b.author_id,
        AVG(r.rating) AS avg_rating,
        COUNT(r.rating_id) AS num_ratings
    FROM 
        books b
    JOIN 
        ratings r ON b.book_id = r.book_id
    GROUP BY 
        b.book_id, b.author_id
    HAVING 
        COUNT(r.rating_id) >= 50
)

SELECT 
    a.author,
    ROUND(AVG(la.avg_rating), 2) AS media_geral,
    COUNT(la.book_id) AS livros_qualificados,
    ROUND(AVG(la.num_ratings)) AS avg_avaliacoes
FROM 
    authors a
JOIN 
    livros_avaliados la ON a.author_id = la.author_id
GROUP BY 
    a.author
ORDER BY 
    media_geral DESC
LIMIT 5;
"""

pd.io.sql.read_sql(query, con=engine)

,author,media_geral,livros_qualificados,avg_avaliacoes
0,J.K. Rowling/Mary GrandPré,4.28,4,78.0
1,Markus Zusak/Cao Xuân Việt Khương,4.26,1,53.0
2,J.R.R. Tolkien,4.26,2,81.0
3,Louisa May Alcott,4.19,1,52.0
4,Rick Riordan,4.08,1,62.0


* Acima podemos ver os Top 5 melhores avaliados e tambem com a maior qauntidadade de avaliações por livros, algo importate na hora de decidir qual os livros e autores devemos escolher para focar o marketing.


<div class="alert alert-success">
<strong>Comentário do revisor v1</strong>

A questão está correta!

</div>

# Encontre o número médio de avaliações entre usuários que classificaram mais do que 50 livros.

In [16]:
query = """
WITH 
usuarios_ativos AS (
    -- Passo 1: Identificar usuários com mais de 50 classificações
    SELECT 
        username
    FROM 
        ratings
    GROUP BY 
        username
    HAVING 
        COUNT(rating_id) > 50
),

reviews_usuarios_ativos AS (
    -- Passo 2: Contar reviews desses usuários ativos
    SELECT 
        username,
        COUNT(review_id) AS num_reviews
    FROM 
        reviews
    WHERE 
        username IN (SELECT username FROM usuarios_ativos)
    GROUP BY 
        username
)
-- Passo 3: Calcular estatísticas das reviews
SELECT 
    ROUND(AVG(num_reviews), 2) AS media_reviews_por_usuario,
    MIN(num_reviews) AS minimo_reviews,
    MAX(num_reviews) AS maximo_reviews,
    COUNT(*) AS total_usuarios_analisados,
    SUM(num_reviews) AS total_reviews
FROM 
    reviews_usuarios_ativos;
"""

resultado = pd.io.sql.read_sql(query, con=engine)
resultado

,media_reviews_por_usuario,minimo_reviews,maximo_reviews,total_usuarios_analisados,total_reviews
0,24.33,18,28,6,146.0



<div class="alert alert-danger">
<strong>Comentário do revisor</strong>

Quase isso!
    
Aqui precisamos usar duas tabelas: `ratings` e `reviews`.
    
Primeiro precisamos identificar os usuários que classificaram mais de 50 livros, usando a tabela `ratings`.
    
Depois, **a partir desses usuários**, precisamos **contar** o número médio de avaliações, usando a tabela `reviews`. Ou seja, devemos refazer essa última query considerando apenas os usuários filtrados na anterior.
    
Com essa dica, acredito que seja possível para você escrever uma _query_ que irá atender perfeitamente à solicitação do projeto.

</div>

<div class="alert alert-success">
<strong>Comentário do revisor v2</strong>

Valeu pelo ajuste!
</div>


# Top 20 livros com melhores avaliados.

In [17]:
query = """
SELECT 
    b.book_id,
    b.title,
    a.author,
    COUNT(r.rating_id) AS total_avaliacoes,
    ROUND(AVG(r.rating), 2) AS media_avaliacao
FROM 
    books b
JOIN 
    ratings r ON b.book_id = r.book_id
JOIN 
    authors a ON b.author_id = a.author_id
GROUP BY 
    b.book_id, b.title, a.author
HAVING 
    COUNT(r.rating_id) >= 10  -- Considerando apenas livros com pelo menos 10 avaliações
ORDER BY 
    media_avaliacao DESC,
    total_avaliacoes DESC
LIMIT 20;
"""

top_livros = pd.io.sql.read_sql(query, con=engine)
top_livros

,book_id,title,author,total_avaliacoes,media_avaliacao
0,581,Snow Flower and the Secret Fan,Lisa See,11,4.64
1,74,And Then There Were None,Agatha Christie,22,4.45
2,284,Gone with the Wind,Margaret Mitchell,35,4.43
3,422,Matilda,Roald Dahl/Quentin Blake,19,4.42
4,302,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,82,4.41
5,722,The Fellowship of the Ring (The Lord of the Ri...,J.R.R. Tolkien,74,4.39
6,15,A Clash of Kings (A Song of Ice and Fire #2),George R.R. Martin,23,4.39
7,830,The Poisonwood Bible,Barbara Kingsolver,22,4.36
8,784,The Lorax,Dr. Seuss,10,4.30
9,286,Goodnight Moon,Margaret Wise Brown/Clement Hurd,10,4.30


# Conclusões finais.

Para uma melhor acertividade a melhor opção e focar os esforços com o marketing nos livros com as melhores avaliações e quantidade maior de avaliações, asssim chamando a atenção do publico e trazendo mais usuarios a plataforma.

"O foco estratégico em obras altamente avaliadas e populares demonstra ser a alavanca mais eficiente para crescimento, combinando:

Credibilidade social comprovada

Alto potencial de engajamento

Efeito rede comprovado
Garantindo assim aquisição de usuários com menor CAC e maior LTV.